# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-75f3f9a2ca-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Companies Workflow Series ("Business People in Germany") 

Consider the following exploratory information need:

> You are investigating important business people in Germany

wd:Q2462658 ## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`      | country       | predicate |
| `wdt:P27`      | citizenship   | predicate |
| `wdt:P106`     | occupation   | predicate |
| `wd:Q183`      | Germany      | node      |
| `wd:Q2462658`  | manager      | node      |
| `wd:Q40966`    | Opel         | node      |
| `wd:Q56509715` | Michael Lohscheller | node |
| `wd:Q57479`    | Adam Opel    | node |



Also consider

```

?p wdt:P27 wd:Q183  . 
?p wdt:P106 wd:Q2462658  . 

```

is the BGP to retrieve all **german managers**

## Workload Goals

1. Identify the BGP for obtaining C.E.O., managers, directors, or founders of German companies

2. Identify the BGP to retrieve gender and profession of people and industry of companies

3. Are there german companies with C.E.O., managers, or founders that are not German?

4. Are there people related to multiple german companies?

5. Analyze the number of business people per role, type of company, and gender
 
   5.1 How many people for each role and gender are there in Germany?
   
   5.2 Are there companies with multiple german people having important roles?
   
   5.3 In which sectors are important german business people working?
   
   5.4 Are there german business people related to non german companies? How many?


In [2]:
# start your workflow here

In [5]:
queryString = """
SELECT COUNT(*)
WHERE { 

    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q2462658  . 

    
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '829')]


1

In [6]:
#return all german managers

queryString = """
SELECT ?name
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q2462658  . 
    ?p <http://schema.org/name> ?name .
} 
ORDER BY ?name
"""

print("Results")
run_query(queryString)

Results
[('name', 'Alan Hippe')]
[('name', 'Albert Behn')]
[('name', 'Albert Caspers')]
[('name', 'Albert Vietor')]
[('name', 'Albrecht Schmidt')]
[('name', 'Albrecht Schultz')]
[('name', 'Alexander Dibelius')]
[('name', 'Alexander Selent')]
[('name', 'Alexander Wiedenhoff')]
[('name', 'Alexandra von Rehlingen')]
[('name', 'Alfred Dänzer')]
[('name', 'Ali Haurand')]
[('name', 'Alice Schardt-Roßmann')]
[('name', 'Alois Alzheimer')]
[('name', 'Amadeus Naumann')]
[('name', 'Anastassia Lauterbach')]
[('name', 'Andrea Schauer')]
[('name', 'Andreas Bartl')]
[('name', 'Andreas Bierwirth')]
[('name', 'Andreas Penk')]
[('name', 'Andreas Zitzewitz')]
[('name', 'Andreas von Arnim')]
[('name', 'André Zeug')]
[('name', 'Andy Mattes')]
[('name', 'Angela Titzrath')]
[('name', 'Anja Polzer')]
[('name', 'Antonio Knauth')]
[('name', 'Ariane Reinhart')]
[('name', 'Armin Zitzmann')]
[('name', 'Arno Bohn')]
[('name', 'Arno Eckardt')]
[('name', 'Arnold E. Weber')]
[('name', 'Arthur Thiele')]
[('name', 'Arth

829

In [7]:
#return the roles of people in german

queryString = """
SELECT DISTINCT ?role ?roleName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 ?role  . 
    ?role <http://schema.org/name> ?roleName .
} 
ORDER BY ?roleName
"""

print("Results")
run_query(queryString)

Results
[('role', 'http://www.wikidata.org/entity/Q15936497'), ('roleName', 'Africanist')]
[('role', 'http://www.wikidata.org/entity/Q17123348'), ('roleName', 'Agriculteur')]
[('role', 'http://www.wikidata.org/entity/Q42677192'), ('roleName', 'Alberte')]
[('role', 'http://www.wikidata.org/entity/Q42331263'), ('roleName', 'American football coach')]
[('role', 'http://www.wikidata.org/entity/Q19204627'), ('roleName', 'American football player')]
[('role', 'http://www.wikidata.org/entity/Q592996'), ('roleName', 'Amtmann')]
[('role', 'http://www.wikidata.org/entity/Q480298'), ('roleName', 'Amtsarzt')]
[('role', 'http://www.wikidata.org/entity/Q481183'), ('roleName', 'Amtshauptmann')]
[('role', 'http://www.wikidata.org/entity/Q3409375'), ('roleName', 'Anglican priest')]
[('role', 'http://www.wikidata.org/entity/Q613087'), ('roleName', 'Anwalt')]
[('role', 'http://www.wikidata.org/entity/Q620573'), ('roleName', 'Arabist')]
[('role', 'http://www.wikidata.org/entity/Q52231330'), ('roleName', '

3115

* wd:Q19435686 'founder'
* wd:Q484876'chief executive officer'
* wd:Q1162163 'director'
* wd:Q205375' 'inventor
* wd:Q703534  'employee'

In [13]:
#check why we have two entity director
#subclass of artist
queryString = """
SELECT DISTINCT ?pred ?predName ?obj ?objName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q3455803. 
    wd:Q3455803 ?pred ?obj.
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1687'), ('predName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P57'), ('objName', 'director')]
[('pred', 'http://www.wikidata.org/prop/direct/P6186'), ('predName', 'category for eponymous categories'), ('obj', 'http://www.wikidata.org/entity/Q8104450'), ('objName', 'Category:Wikipedia categories named after filmmakers')]
[('pred', 'http://www.wikidata.org/prop/direct/P425'), ('predName', 'field of this occupation'), ('obj', 'http://www.wikidata.org/entity/Q2137554'), ('objName', 'direction')]
[('pred', 'http://www.wikidata.org/prop/direct/P101'), ('predName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q2137554'), ('objName', 'direction')]
[('pred', 'http://www.wikidata.org/prop/direct/P101'), ('predName', 'field of work'), ('obj', 'http://www.wikidata.org/entity/Q2607241'), ('objName', 'mise en scène')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of'), ('obj', 'ht

11

In [30]:
#check why we have two entity director
# wd:Q1162163 this is the entity for the right director
queryString = """
SELECT DISTINCT ?pred ?predName ?obj ?objName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q1162163. 
    wd:Q1162163 ?pred ?obj.
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1687'), ('predName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P1037'), ('objName', 'director / manager')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q12737077'), ('objName', 'occupation')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q28640'), ('objName', 'profession')]
[('pred', 'http://www.wikidata.org/prop/direct/P31'), ('predName', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q4164871'), ('objName', 'position')]
[('pred', 'http://www.wikidata.org/prop/direct/P2389'), ('predName', 'organization directed by the office or position'), ('obj', 'http://www.wikidata.org/entity/Q21002644'), ('objName', 'Cultural Center of Spain in Malabo')]
[('pred', 'http://www.wikidata.org/prop/direct/P2389'), ('predName', 'organization directed by the office or 

14

In [31]:
# since I have Adam Opel that is a founder of Opel, I retrieve information following the founder path.
# incoming edges in founder
queryString = """
SELECT DISTINCT ?predName ?subName ?sub
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q19435686. 
    ?sub ?pred wd:Q19435686.
    ?pred <http://schema.org/name> ?predName .
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY ?subName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'occupation'), ('subName', 'Abner Jones'), ('sub', 'http://www.wikidata.org/entity/Q4668140')]
[('predName', 'occupation'), ('subName', 'Adrien-Aurélien Hébrard'), ('sub', 'http://www.wikidata.org/entity/Q106350089')]
[('predName', 'occupation'), ('subName', 'Albert Räthel'), ('sub', 'http://www.wikidata.org/entity/Q51012313')]
[('predName', 'occupation'), ('subName', 'Alexander McCaine'), ('sub', 'http://www.wikidata.org/entity/Q55720150')]
[('predName', 'occupation'), ('subName', 'Alexis Rudier'), ('sub', 'http://www.wikidata.org/entity/Q56246396')]
[('predName', 'occupation'), ('subName', 'Alfred Zöttl'), ('sub', 'http://www.wikidata.org/entity/Q105414520')]
[('predName', 'occupation'), ('subName', 'Andreas Rive'), ('sub', 'http://www.wikidata.org/entity/Q55126136')]
[('predName', 'occupation'), ('subName', 'Ann Teresa Matthews'), ('sub', 'http://www.wikidata.org/entity/Q55720374')]
[('predName', 'occupation'), ('subName', 'Anna Jarvis'), ('sub', 'http://www.wi

168

In [37]:
# return the information about Adam Opel
queryString = """
SELECT DISTINCT ?pred ?predName ?objName
WHERE { 
    wd:Q57479 ?pred ?obj.
  
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P40'), ('predName', 'child'), ('objName', 'Carl von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P40'), ('predName', 'child'), ('objName', 'Heinrich von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P40'), ('predName', 'child'), ('objName', 'Wilhelm von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P40'), ('predName', 'child'), ('objName', 'Ludwig Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P40'), ('predName', 'child'), ('objName', 'Friedrich Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P27'), ('predName', 'country of citizenship'), ('objName', 'Germany')]
[('pred', 'http://www.wikidata.org/prop/direct/P1343'), ('predName', 'described by source'), ('objName', 'Obalky knih.cz')]
[('pred', 'http://www.wikidata.org/prop/direct/P734'), ('predName', 'family name'), ('objName', 'Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P735'), ('predName', 'given name'), ('objName', 'Adam')]


22

In [41]:
# since I have Adam Opel that is a founder of Opel, I retrieve information following the founder path.
queryString = """
SELECT DISTINCT ?predName ?objName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q19435686. 
    wd:Q19435686 ?pred ?obj.
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'field of this occupation'), ('objName', 'metal casting')]
[('predName', 'instance of'), ('objName', 'profession')]
[('predName', 'subclass of'), ('objName', 'artisan')]
[('predName', 'subclass of'), ('objName', 'metalsmith')]
[('predName', "topic's main category"), ('objName', 'Category:Foundrymen')]


5

In [34]:
# return the role of Adam Opel if there exists
queryString = """
SELECT DISTINCT ?role ?roleName 
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 ?role. 
    ?role ?pred wd:Q57479.
    ?role <http://schema.org/name> ?roleName .
} 
ORDER BY ?subName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [44]:
# return the object connected to the founder
queryString = """
SELECT DISTINCT ?pred ?predName ?obj ?objName 
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q19435686. 
    wd:Q19435686 wdt:P31 ?profession.
    ?profession ?pred ?obj.
    ?obj <http://schema.org/name> ?objName .
    ?pred <http://schema.org/name> ?predName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P1687'), ('predName', 'Wikidata property'), ('obj', 'http://www.wikidata.org/entity/P106'), ('objName', 'occupation')]
[('pred', 'http://www.wikidata.org/prop/direct/P1343'), ('predName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q101314624'), ('objName', 'Lean Logic: A Dictionary for the Future and How to Survive It')]
[('pred', 'http://www.wikidata.org/prop/direct/P1343'), ('predName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q19180675'), ('objName', 'Small Brockhaus and Efron Encyclopedic Dictionary')]
[('pred', 'http://www.wikidata.org/prop/direct/P1343'), ('predName', 'described by source'), ('obj', 'http://www.wikidata.org/entity/Q4173137'), ('objName', 'Jewish Encyclopedia of Brockhaus and Efron')]
[('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('predName', 'different from'), ('obj', 'http://www.wikidata.org/entity/Q828803'), ('objName', 'job title')]
[('pred', 'http://ww

15

In [56]:
# return the object connected to the founder
queryString = """
SELECT DISTINCT ?jobName ?predName ?objName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q19435686. 
    wd:Q19435686 wdt:P31/wdt:P279 ?job.
    ?job ?pred ?obj.
  
    ?pred <http://schema.org/name> ?predName .
    ?job <http://schema.org/name> ?jobName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?jobName
"""

print("Results")
run_query(queryString)

Results
[('jobName', 'job'), ('predName', "topic's main template"), ('objName', 'Template:Aspects of occupations')]
[('jobName', 'job'), ('predName', "topic's main template"), ('objName', 'Template:Aspects of jobs')]
[('jobName', 'job'), ('predName', 'has effect'), ('objName', 'occupational stress')]
[('jobName', 'job'), ('predName', 'Wikidata property'), ('objName', 'position held')]
[('jobName', 'job'), ('predName', 'different from'), ('objName', 'employment')]
[('jobName', 'job'), ('predName', 'different from'), ('objName', 'job vacancy')]
[('jobName', 'job'), ('predName', 'subclass of'), ('objName', 'work')]
[('jobName', 'job'), ('predName', 'instance of'), ('objName', 'social status')]
[('jobName', 'job'), ('predName', 'said to be the same as'), ('objName', 'work')]
[('jobName', 'occupation'), ('predName', 'Wikidata property'), ('objName', 'field of this occupation')]
[('jobName', 'occupation'), ('predName', 'Wikidata property'), ('objName', 'occupation')]
[('jobName', 'occupation

30

In [62]:
# return the information about Adam Opel (outgoing edges)
# occupation' 'inventor
queryString = """
SELECT DISTINCT ?pred ?predName ?objName ?obj
WHERE { 
    wd:Q57479 ?pred ?obj.
  
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?jobName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P735'), ('predName', 'given name'), ('objName', 'Adam'), ('obj', 'http://www.wikidata.org/entity/Q347181')]
[('pred', 'http://www.wikidata.org/prop/direct/P1830'), ('predName', 'owner of'), ('objName', 'Opel'), ('obj', 'http://www.wikidata.org/entity/Q40966')]
[('pred', 'http://www.wikidata.org/prop/direct/P106'), ('predName', 'occupation'), ('objName', 'entrepreneur'), ('obj', 'http://www.wikidata.org/entity/Q131524')]
[('pred', 'http://www.wikidata.org/prop/direct/P27'), ('predName', 'country of citizenship'), ('objName', 'Germany'), ('obj', 'http://www.wikidata.org/entity/Q183')]
[('pred', 'http://www.wikidata.org/prop/direct/P1412'), ('predName', 'languages spoken, written or signed'), ('objName', 'German'), ('obj', 'http://www.wikidata.org/entity/Q188')]
[('pred', 'http://www.wikidata.org/prop/direct/P106'), ('predName', 'occupation'), ('objName', 'inventor'), ('obj', 'http://www.wikidata.org/entity/Q205375')]
[('pred', 'http:

22

In [67]:
# return the information about Adam Opel (outgoing edges)
# occupation' 'inventor
queryString = """
SELECT DISTINCT ?predName ?objName
WHERE { 
    wd:Q57479 wdt:P106 ?profession.
    ?profession ?pred ?obj.
  
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?jobName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'subclass of'), ('objName', 'skilled worker')]
[('predName', 'subclass of'), ('objName', 'manual worker')]
[('predName', 'field of work'), ('objName', 'entrepreneurship')]
[('predName', 'Wikidata property'), ('objName', 'discoverer or inventor')]
[('predName', "topic's main category"), ('objName', 'Category:Inventors')]
[('predName', 'field of this occupation'), ('objName', 'invention')]
[('predName', 'described by source'), ('objName', 'Armenian Soviet Encyclopedia')]
[('predName', 'instance of'), ('objName', 'profession')]
[('predName', 'instance of'), ('objName', 'position')]
[('predName', 'subclass of'), ('objName', 'businessperson')]
[('predName', 'subclass of'), ('objName', 'engineer')]
[('predName', 'said to be the same as'), ('objName', 'entrepreneur')]
[('predName', 'different from'), ('objName', 'entrepreneur')]
[('predName', 'subclass of'), ('objName', 'creator')]
[('predName', 'field of this occupation'), ('objName', 'machining')]
[('predName', "topic'

17

In [63]:
# return the information about Adam Opel (incoming edges)
# wdt:P112' 'Opel'
queryString = """
SELECT DISTINCT ?pred ?predName ?subName
WHERE { 
    ?sub ?pred wd:Q57479 .
  
    ?pred <http://schema.org/name> ?predName .
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P301'), ('predName', "category's main topic"), ('subName', 'Category:Adam Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P22'), ('predName', 'father'), ('subName', 'Carl von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P22'), ('predName', 'father'), ('subName', 'Heinrich von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P22'), ('predName', 'father'), ('subName', 'Wilhelm von Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P22'), ('predName', 'father'), ('subName', 'Ludwig Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P22'), ('predName', 'father'), ('subName', 'Friedrich Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P112'), ('predName', 'founded by'), ('subName', 'Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P138'), ('predName', 'named after'), ('subName', 'Opel')]
[('pred', 'http://www.wikidata.org/prop/direct/P138'), ('predName', 'named after'), ('subName', 'Opel Adam')

11

In [78]:
# investigates the Opel company
queryString = """
SELECT DISTINCT ?predName ?pred ?objName 
WHERE { 
    wd:Q40966 ?pred ?obj.
    ?pred <http://schema.org/name> ?predName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'board member'), ('pred', 'http://www.wikidata.org/prop/direct/P3320'), ('objName', 'Fritz von Opel')]
[('predName', 'board member'), ('pred', 'http://www.wikidata.org/prop/direct/P3320'), ('objName', 'Wilhelm von Opel')]
[('predName', 'board member'), ('pred', 'http://www.wikidata.org/prop/direct/P3320'), ('objName', 'Friedrich Opel')]
[('predName', 'chief executive officer'), ('pred', 'http://www.wikidata.org/prop/direct/P169'), ('objName', 'Uwe Hochgeschurtz')]
[('predName', 'country'), ('pred', 'http://www.wikidata.org/prop/direct/P17'), ('objName', 'Germany')]
[('predName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('objName', 'Buick Opel')]
[('predName', 'director / manager'), ('pred', 'http://www.wikidata.org/prop/direct/P1037'), ('objName', 'Michael Lohscheller')]
[('predName', 'founded by'), ('pred', 'http://www.wikidata.org/prop/direct/P112'), ('objName', 'Adam Opel')]
[('predName', 'headquarters location'), ('pred', 'http

28

In [10]:
# checks if everything works with the initial BGP
queryString = """
SELECT DISTINCT ?pName ?p 
WHERE { 
    ?manager wdt:P27 wd:Q183  . 
    ?manager wdt:P106 wd:Q2462658  .
    ?obj ?p ?manager.
    ?p <http://schema.org/name> ?pName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'defendant'), ('p', 'http://www.wikidata.org/prop/direct/P1591')]
[('pName', 'director / manager'), ('p', 'http://www.wikidata.org/prop/direct/P1037')]
[('pName', 'relative'), ('p', 'http://www.wikidata.org/prop/direct/P1038')]
[('pName', 'founded by'), ('p', 'http://www.wikidata.org/prop/direct/P112')]
[('pName', 'owned by'), ('p', 'http://www.wikidata.org/prop/direct/P127')]
[('pName', 'winner'), ('p', 'http://www.wikidata.org/prop/direct/P1346')]
[('pName', 'named after'), ('p', 'http://www.wikidata.org/prop/direct/P138')]
[('pName', 'cast member'), ('p', 'http://www.wikidata.org/prop/direct/P161')]
[('pName', 'producer'), ('p', 'http://www.wikidata.org/prop/direct/P162')]
[('pName', 'curator'), ('p', 'http://www.wikidata.org/prop/direct/P1640')]
[('pName', 'chief executive officer'), ('p', 'http://www.wikidata.org/prop/direct/P169')]
[('pName', 'depicts'), ('p', 'http://www.wikidata.org/prop/direct/P180')]
[('pName', 'doctoral advisor'), ('p', 'http://www.wikidat

36

In [15]:
# checks if everything works with the initial BGP
queryString = """
SELECT DISTINCT ?pName ?p 
WHERE { 
    ?manager wdt:P27 wd:Q183  . 
    ?manager wdt:P106 wd:Q2462658  .
    ?manager ?p ?obj.
    ?p <http://schema.org/name> ?pName .
    ?obj <http://schema.org/name> ?objName .
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'academic degree'), ('p', 'http://www.wikidata.org/prop/direct/P512')]
[('pName', 'academic major'), ('p', 'http://www.wikidata.org/prop/direct/P812')]
[('pName', 'academic minor'), ('p', 'http://www.wikidata.org/prop/direct/P811')]
[('pName', 'affiliation'), ('p', 'http://www.wikidata.org/prop/direct/P1416')]
[('pName', 'allegiance'), ('p', 'http://www.wikidata.org/prop/direct/P945')]
[('pName', 'award received'), ('p', 'http://www.wikidata.org/prop/direct/P166')]
[('pName', 'candidacy in election'), ('p', 'http://www.wikidata.org/prop/direct/P3602')]
[('pName', 'cause of death'), ('p', 'http://www.wikidata.org/prop/direct/P509')]
[('pName', 'child'), ('p', 'http://www.wikidata.org/prop/direct/P40')]
[('pName', 'conflict'), ('p', 'http://www.wikidata.org/prop/direct/P607')]
[('pName', 'convicted of'), ('p', 'http://www.wikidata.org/prop/direct/P1399')]
[('pName', 'copyright representative'), ('p', 'http://www.wikidata.org/prop/direct/P6275')]
[('pName', 'copyright s

72

Important relationship:<br>
* wdt:P169 chief executive officer
* wdt:P1037 director/manager
* wdt:P3320 board member
* wdt:112 founded by

In [3]:
# checks if everything works with the initial BGP
queryString = """
SELECT DISTINCT ?pName ?companyName
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q2462658  .
    ?company wdt:P1037 ?p.
    ?p <http://schema.org/name> ?pName .
    ?company <http://schema.org/name> ?companyName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Leonhard Fischer'), ('companyName', 'BHF Kleinwort Benson Group')]
[('pName', 'Andreas Bierwirth'), ('companyName', 'UPC Austria')]
[('pName', 'Andreas Bierwirth'), ('companyName', 'Magenta Telekom')]
[('pName', 'Hans-Holger Albrecht'), ('companyName', 'Deezer')]
[('pName', 'Otto Arndt'), ('companyName', 'Deutsche Reichsbahn (GDR)')]
[('pName', 'Johannes Teyssen'), ('companyName', 'E.ON SE')]
[('pName', 'Siegfried Jaschinski'), ('companyName', 'Heidelberger Druckmaschinen')]
[('pName', 'Jutta Cordt'), ('companyName', 'Federal Office for Migration and Refugees')]
[('pName', 'Wolfgang Kirsch'), ('companyName', 'Bausparkasse Schwäbisch Hall AG')]
[('pName', 'Gotthard Haug'), ('companyName', 'Teleplan')]
[('pName', 'Wilhelm Fitzner'), ('companyName', 'Deutsche Reichsbahn (GDR)')]
[('pName', 'Fritz Harders'), ('companyName', 'Dortmund-Hörder Hüttenunion')]
[('pName', 'Wolfgang Marzin'), ('companyName', 'Messe Frankfurt')]
[('pName', 'Willi Kreikemeyer'), ('companyName', 

61

In [17]:
# checks if everything works with the initial BGP
queryString = """
SELECT DISTINCT ?pName ?companyName ?company
WHERE { 
    ?p wdt:P27 wd:Q183  . 
    ?p wdt:P106 wd:Q2462658  .
    ?company wdt:P112 ?p.
    ?p <http://schema.org/name> ?pName .
    ?company <http://schema.org/name> ?companyName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Tilo Wolff'), ('companyName', 'Hall of Sermon'), ('company', 'http://www.wikidata.org/entity/Q567947')]
[('pName', 'Richard Lattorf'), ('companyName', 'Deutsche Naturasphaltgesellschaft m.b.H.'), ('company', 'http://www.wikidata.org/entity/Q68283367')]
[('pName', 'Karl-Peter Naumann'), ('companyName', 'Verkehrsclub Deutschland'), ('company', 'http://www.wikidata.org/entity/Q2516162')]
[('pName', 'Robert Gentz'), ('companyName', 'Zalando SE'), ('company', 'http://www.wikidata.org/entity/Q136570')]
[('pName', 'Karl-Josef Neukirchen'), ('companyName', 'Kajo Neukirchen Group'), ('company', 'http://www.wikidata.org/entity/Q26272046')]
[('pName', 'Helmut Hoffer von Ankershoffen'), ('companyName', 'Neofonie'), ('company', 'http://www.wikidata.org/entity/Q1977495')]


6

In [108]:
# investigates the Opel company
queryString = """
SELECT DISTINCT  ?predName ?subName
WHERE { 
    ?sub ?pred wd:Q40966.
    ?pred <http://schema.org/name> ?predName .
    ?sub <http://schema.org/name> ?subName .
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'brand'), ('subName', 'Opel Zafira')]
[('predName', 'brand'), ('subName', 'Opel Sintra')]
[('predName', 'brand'), ('subName', 'Opel 13/30 PS')]
[('predName', 'brand'), ('subName', 'Opel 8/40 PS')]
[('predName', 'brand'), ('subName', 'Opel Commodore B')]
[('predName', 'brand'), ('subName', 'Opel Commodore A')]
[('predName', 'brand'), ('subName', 'Opel P4')]
[('predName', 'brand'), ('subName', 'Opel Kadett')]
[('predName', 'brand'), ('subName', 'Opel 1.2 litre')]
[('predName', 'brand'), ('subName', 'Opel Ascona')]
[('predName', 'brand'), ('subName', 'Opel Vectra')]
[('predName', 'brand'), ('subName', 'Opel Insignia')]
[('predName', 'brand'), ('subName', 'Opel Tigra Twintop')]
[('predName', 'brand'), ('subName', 'Opel Tigra')]
[('predName', 'brand'), ('subName', 'Isuzu Amigo')]
[('predName', 'brand'), ('subName', 'Opel Ampera')]
[('predName', 'brand'), ('subName', 'Opel Karl')]
[('predName', 'brand'), ('subName', 'Opel Agila')]
[('predName', 'brand'), ('subName', 'Op

360

In [ ]:
# return the managers in the big company of Opel
queryString = """
SELECT DISTINCT  ?personName ?bigCompanyName
WHERE { 
    ?person wdt:P27 wd:Q183. 
    ?person wdt:P106 ?roles. 
    wd:Q40966 wdt:P1037 ?person;
        wdt:P17 wd:Q183;
        wdt:P127 ?bigCompany.

   
    ?person <http://schema.org/name> ?personName.
    ?bigCompany <http://schema.org/name> ?bigCompanyName.
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Michael Lohscheller'), ('bigCompanyName', 'Stellantis')]


1

In [4]:
# return the managers in the big company of Opel
queryString = """
SELECT DISTINCT  ?personName ?bigCompanyName
WHERE { 
    ?person wdt:P27 wd:Q183. 
    ?person wdt:P106 wd:Q19435686. 
    wd:Q40966 ?pred ?person;
        wdt:P17 wd:Q183;
        wdt:P127 ?bigCompany.

   
    ?person <http://schema.org/name> ?personName.
    ?bigCompany <http://schema.org/name> ?bigCompanyName.
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [8]:
# return the managers in the big company of Opel
queryString = """
SELECT DISTINCT  ?personName ?bigCompanyName
WHERE { 
    ?person wdt:P27 wd:Q183. 
    ?person wdt:P106 wd:Q19435686. 
    ?person ?pred wd:Q40966.
     wd:Q40966 wdt:P17 wd:Q183;
        wdt:P127 ?bigCompany.

   
    ?person <http://schema.org/name> ?personName.
    ?bigCompany <http://schema.org/name> ?bigCompanyName.
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [21]:
# return the predicates connected to the company
queryString = """
SELECT DISTINCT ?pName ?p
WHERE { 
    ?manager wdt:P27 wd:Q183  . 
    ?manager wdt:P106 wd:Q2462658  .
    ?company wdt:P112 ?manager.
    ?company ?p ?obj.
    ?p <http://schema.org/name> ?pName .
    ?company <http://schema.org/name> ?companyName .
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pName', 'Alexa rank'), ('p', 'http://www.wikidata.org/prop/direct/P1661')]
[('pName', 'Commons category'), ('p', 'http://www.wikidata.org/prop/direct/P373')]
[('pName', 'Crunchbase organization ID'), ('p', 'http://www.wikidata.org/prop/direct/P2088')]
[('pName', 'Discogs label ID'), ('p', 'http://www.wikidata.org/prop/direct/P1955')]
[('pName', 'EU Research participant ID'), ('p', 'http://www.wikidata.org/prop/direct/P5785')]
[('pName', 'EU Transparency Register ID'), ('p', 'http://www.wikidata.org/prop/direct/P2657')]
[('pName', 'EU VAT number'), ('p', 'http://www.wikidata.org/prop/direct/P3608')]
[('pName', 'Encyclopaedia Metallum label ID'), ('p', 'http://www.wikidata.org/prop/direct/P8166')]
[('pName', 'Facebook ID'), ('p', 'http://www.wikidata.org/prop/direct/P2013')]
[('pName', 'Freebase ID'), ('p', 'http://www.wikidata.org/prop/direct/P646')]
[('pName', 'GND ID'), ('p', 'http://www.wikidata.org/prop/direct/P227')]
[('pName', 'GitHub username'), ('p', 'http://www.wikid

59

*******
# Query 1
Identify the BGP for obtaining C.E.O., managers, directors, or founders of German companies.


In [27]:
# return the C.E.O., managers, directors, or founders of German companies and their companies
queryString = """
SELECT DISTINCT ?managerName ?companyName
WHERE { 
    ?manager wdt:P106 wd:Q2462658. 
    ?company wdt:P1037|wdt:P169|wdt:P112 ?manager;
        wdt:P17 wd:Q183.
        
    ?manager <http://schema.org/name> ?managerName.
    ?company <http://schema.org/name> ?companyName.
} 
ORDER BY ?managerName
"""

print("Results")
run_query(queryString)

Results
[('managerName', 'Aldo Belloni'), ('companyName', 'Linde')]
[('managerName', 'Angela Titzrath'), ('companyName', 'HHLA')]
[('managerName', 'Armin Zitzmann'), ('companyName', 'Nürnberger Versicherung')]
[('managerName', 'Axel Stepken'), ('companyName', 'TÜV Süd')]
[('managerName', 'Carola Scheidling'), ('companyName', 'Dr. Ebel Heinrich-Heine-Clinic')]
[('managerName', 'Carsten Spohr'), ('companyName', 'Lufthansa')]
[('managerName', 'Christian Göke'), ('companyName', 'Messe Berlin')]
[('managerName', 'Derk Haank'), ('companyName', 'Springer')]
[('managerName', 'Dirk Stenkamp'), ('companyName', 'TÜV Nord')]
[('managerName', 'Dirk Stenkamp'), ('companyName', 'Carl Zeiss NTS')]
[('managerName', 'Dr. Markus Steilemann'), ('companyName', 'Covestro')]
[('managerName', 'Elmar Degenhart'), ('companyName', 'Continental AG')]
[('managerName', 'Ernst Georg Stöckl'), ('companyName', 'AEG')]
[('managerName', 'Eva Wunsch-Weber'), ('companyName', 'Frankfurter Volksbank')]
[('managerName', 'Fer

110

> Initially, I've searched in a bad way the information. So, I've start from Adam Opel and check if there exists in the founder role, but I wasn't able to find him. So, I've taken the single entity 'Adam Opel' and check the predicates. Also in this case, I haven't found anything useful. So, I've put directly Adam Opel connected to the role and the query was empty.<br>
> I've restarted from the initial BGP and searched the predicates and objects connected to the managers. Then, I've found:
> * wdt:P1037 *director/manager*
> * wdt:P169 *executive*
> * wdt:P112 *founded by*<br>
>
> Then, I've checked the information from the company in order to retrieve the country.
******

In [14]:
# return industry of the Opel company
queryString = """
SELECT DISTINCT ?predName ?subName
WHERE { 
     wd:Q40966 wdt:P452 ?industry.
     ?sub ?pred ?industry.

    ?pred <http://schema.org/name> ?predName.
    ?sub <http://schema.org/name> ?subName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'academic degree'), ('subName', 'Rima Varzhapetyan-Feller')]
[('predName', 'academic major'), ('subName', 'David Austin Hagen')]
[('predName', 'academic major'), ('subName', 'Edmund Rumpler')]
[('predName', 'academic major'), ('subName', 'Jayaveera Pandian Muthusamy')]
[('predName', 'academic major'), ('subName', 'Qibo Li')]
[('predName', 'academic major'), ('subName', 'Minsuk Kong')]
[('predName', 'academic major'), ('subName', 'Omid Saber')]
[('predName', 'academic major'), ('subName', 'Taolue Zhang')]
[('predName', 'academic major'), ('subName', 'Chun-Wei Yao')]
[('predName', 'academic major'), ('subName', 'Fangming Xiang')]
[('predName', 'academic major'), ('subName', 'Gorka Artola Alberdi')]
[('predName', 'academic major'), ('subName', 'Kevin Andrew Laux')]
[('predName', 'academic major'), ('subName', 'Marouen Hamdi')]
[('predName', 'academic major'), ('subName', 'Shuang Xiao')]
[('predName', 'academic major'), ('subName', 'Ya Wang')]
[('predName', 'academic 

4996

info about sector

In [15]:
# found something concerns the sector of the company
#subclass of
queryString = """
SELECT DISTINCT ?predName ?objName
WHERE { 
     wd:Q40966 wdt:P452 ?industry.
     ?industry ?pred ?obj.

    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.
} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'described by source'), ('objName', 'Great Soviet Encyclopedia (1926–1947)')]
[('predName', 'described by source'), ('objName', 'Armenian Soviet Encyclopedia')]
[('predName', 'different from'), ('objName', 'machine industry')]
[('predName', 'different from'), ('objName', 'design engineering')]
[('predName', 'different from'), ('objName', 'civil engineering')]
[('predName', 'different from'), ('objName', 'structural engineering')]
[('predName', 'different from'), ('objName', 'vehicle construction')]
[('predName', 'has list'), ('objName', 'Glossary of mechanical engineering')]
[('predName', 'instance of'), ('objName', 'academic discipline')]
[('predName', 'instance of'), ('objName', 'economic activity')]
[('predName', 'instance of'), ('objName', 'branch of engineering')]
[('predName', 'instance of'), ('objName', 'industry')]
[('predName', 'practiced by'), ('objName', 'mechanical engineer')]
[('predName', 'product or material produced'), ('objName', 'motor car')]
[('

21

Employee (query 2)

In [20]:
# return the gender, the occupation of the employee person and the industry of the company (with its name) where he/she works (restrict to Opel)
queryString = """
SELECT DISTINCT ?companyName ?rolesName ?personName
WHERE { 
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    wd:Q40966 wdt:P452 ?industry.
    ?person wdt:P108 wd:Q40966.

    ?person <http://schema.org/name> ?personName.
    wdt:P108 <http://schema.org/name> ?rolesName.
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
[('rolesName', 'employer'), ('personName', 'Chris Bangle')]
[('rolesName', 'employer'), ('personName', 'Fritz von Opel')]
[('rolesName', 'employer'), ('personName', 'Joschka Fischer')]
[('rolesName', 'employer'), ('personName', 'Kurt Zimmermann')]
[('rolesName', 'employer'), ('personName', 'Lovelyn Enebechi')]
[('rolesName', 'employer'), ('personName', 'Uwe Hochgeschurtz')]


6

CEO (query 2)

In [21]:
# return the gender, the occupation of the CEO person and the industry of the company (with its name) where he/she works (restrict to Opel)
queryString = """
SELECT DISTINCT ?companyName ?rolesName ?personName
WHERE { 
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    wd:Q40966 wdt:P452 ?industry;
        wdt:P169 ?person.

    ?person <http://schema.org/name> ?personName.
   wdt:P169 <http://schema.org/name> ?rolesName.
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)


Results
[('rolesName', 'chief executive officer'), ('personName', 'Uwe Hochgeschurtz')]


1

Manager (query 2)

In [4]:
# return the gender, the occupation of the manager person and the industry of the company (with its name) where he/she works (restrict to Opel)
queryString = """
SELECT DISTINCT ?companyName ?rolesName ?personName
WHERE { 
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    wd:Q40966 wdt:P452 ?industry;
        wdt:P1037 ?person.

    ?person <http://schema.org/name> ?personName.
   wdt:P1037 <http://schema.org/name> ?rolesName.
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)


Results
[('rolesName', 'director / manager'), ('personName', 'Michael Lohscheller')]


1

Board member (query 2)

In [24]:
# return the gender, the occupation of the board member person and the industry of the company (with its name) where he/she works (restrict to Opel)
queryString = """
SELECT DISTINCT ?companyName ?rolesName ?personName
WHERE { 
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    wd:Q40966 wdt:P452 ?industry;
        wdt:P3320 ?person.

    ?person <http://schema.org/name> ?personName.
   wdt:P3320 <http://schema.org/name> ?rolesName.
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)


Results
[('rolesName', 'board member'), ('personName', 'Friedrich Opel')]
[('rolesName', 'board member'), ('personName', 'Fritz von Opel')]
[('rolesName', 'board member'), ('personName', 'Wilhelm von Opel')]


3

In [14]:
# return the gender, the occupation of the employee person and the industry of the company (with its name) where he/she works
queryString = """
SELECT DISTINCT ?companyName ?rolesName ?personName
WHERE { 
    ?person wdt:P106 wd:Q2462658;
        wdt:P21 ?gender.
    ?company wdt:P452 ?industry.
    ?person wdt:P108 ?company.

    ?company <http://schema.org/name> ?companyName.
    ?person <http://schema.org/name> ?personName.
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
[('companyName', 'Telia Norge'), ('personName', 'Abraham Foss')]
[('companyName', 'Poczta Polska'), ('personName', 'Adam Bernatowicz')]
[('companyName', 'Toei Animation'), ('personName', 'Akemi Ōta')]
[('companyName', 'Mediaset España'), ('personName', 'Alejandro Echevarría Busquet')]
[('companyName', 'Goldman Sachs'), ('personName', 'Alexander Dibelius')]
[('companyName', 'Deutsche Bahn'), ('personName', 'Alexander Doll')]
[('companyName', 'CinemArt'), ('personName', 'Aleš Tříska')]
[('companyName', 'Migros'), ('personName', 'Alfred Gehrig')]
[('companyName', 'Sephora'), ('personName', 'Alluux')]
[('companyName', 'Allianz SE'), ('personName', 'Alois Alzheimer')]
[('companyName', 'Novartis'), ('personName', 'Andreas Clemens')]
[('companyName', 'Österreichische Bundesbahnen'), ('personName', 'Andreas Matthä')]
[('companyName', 'Siemens'), ('personName', 'Andreas Zitzewitz')]
[('companyName', 'Infineon Technologies'), ('personName', 'Andreas Zitzewitz')]
[('companyName', 'Uralsvy

532

In [19]:
# return the occupation the gender and the company name of the managers
queryString = """
SELECT DISTINCT ?pName ?pred ?companyName ?genderName ?occupationName 
WHERE { 
    ?manager wdt:P27 wd:Q183; 
         wdt:P106 wd:Q2462658;
         wdt:P106 ?occupation;
         wdt:P21 ?gender.
    ?company wdt:P1037|wdt:P169|wdt:P112|wdt:P3320 ?manager.
    
    ?occupation <http://schema.org/name> ?occupationName.
    ?gender <http://schema.org/name> ?genderName .
    ?company <http://schema.org/name> ?companyName .
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('companyName', 'Wiki Education Foundation'), ('genderName', 'male'), ('occupationName', 'Wikipedian')]
[('companyName', 'BMW'), ('genderName', 'male'), ('occupationName', 'executive')]
[('companyName', 'Hall of Sermon'), ('genderName', 'male'), ('occupationName', 'trumpeter')]
[('companyName', 'Gallerie degli Uffizi'), ('genderName', 'male'), ('occupationName', 'museum director')]
[('companyName', 'Uffizi Gallery'), ('genderName', 'male'), ('occupationName', 'museum director')]
[('companyName', 'Dortmund-Hörder Hüttenunion'), ('genderName', 'male'), ('occupationName', 'industrialist')]
[('companyName', 'Deutsche Reichsbahn (GDR)'), ('genderName', 'male'), ('occupationName', 'entrepreneur')]
[('companyName', 'E.ON SE'), ('genderName', 'male'), ('occupationName', 'entrepreneur')]
[('companyName', 'Hall of Sermon'), ('genderName', 'male'), ('occupationName', 'manufacturer')]
[('companyName', 'Munich Security Conference'), ('genderName', 'male'), ('occupationName', 'university te

160

***********
# Query 2
Identify the BGP to retrieve gender and profession of people and industry of companies

In [28]:
# return the name of the manager, its occupation, its gender and the company name 
queryString = """
SELECT DISTINCT ?managerName ?companyName ?genderName ?occupationName 
WHERE { 
    ?manager wdt:P106 wd:Q2462658;
         wdt:P106 ?occupation;
         wdt:P21 ?gender.
    ?company wdt:P1037|wdt:P169|wdt:P112|wdt:P3320 ?manager.
    
    ?occupation <http://schema.org/name> ?occupationName.
    ?manager <http://schema.org/name> ?managerName.
    ?gender <http://schema.org/name> ?genderName .
    ?company <http://schema.org/name> ?companyName .
} 
ORDER BY ?genderName
"""

print("Results")
run_query(queryString)

Results
[('managerName', 'Gal Gabai'), ('companyName', '929 - Bible Together'), ('genderName', 'female'), ('occupationName', 'journalist')]
[('managerName', 'Eva Berneke'), ('companyName', 'KMD'), ('genderName', 'female'), ('occupationName', 'businessperson')]
[('managerName', 'Eva Berneke'), ('companyName', 'KMD'), ('genderName', 'female'), ('occupationName', 'engineer')]
[('managerName', 'Galia Wolloch'), ('companyName', "Na'amat"), ('genderName', 'female'), ('occupationName', 'activist')]
[('managerName', 'Melody Harris-Jensbach'), ('companyName', 'Jack Wolfskin'), ('genderName', 'female'), ('occupationName', 'manager')]
[('managerName', 'Galia Wolloch'), ('companyName', "Na'amat"), ('genderName', 'female'), ('occupationName', 'manager')]
[('managerName', 'Hana Šmejkalová'), ('companyName', 'Pardubice Airport'), ('genderName', 'female'), ('occupationName', 'manager')]
[('managerName', 'Gal Gabai'), ('companyName', '929 - Bible Together'), ('genderName', 'female'), ('occupationName',

1656

> Initially, I've checked the predicates connected to the manager. I've found wdt:P21 *gender*. I've also found wdt:P106 *occupation* and then I've searched a way to connected them to company. 
************

In [29]:
# return the gender, the occupation of the person (alive) and the industry of the company (with its name) where he/she works
# it was another solution for query 2
queryString = """
SELECT DISTINCT ?companyName ?industryName ?rolesName ?genderName
WHERE { 
    {
     ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    ?company wdt:P452 ?industry.
    ?person wdt:P108 ?company.

    ?person <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
    ?industry <http://schema.org/name> ?industryName.
    ?gender <http://schema.org/name> ?genderName.
    wdt:P108 <http://schema.org/name> ?rolesName.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    ?company wdt:P452 ?industry;
        wdt:P1037 ?person.

    ?person <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
    ?industry <http://schema.org/name> ?industryName.
    ?gender <http://schema.org/name> ?genderName.
    wdt:P1037 <http://schema.org/name> ?rolesName.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    ?company wdt:P452 ?industry;
        wdt:P169 ?person.

    ?person <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
    ?industry <http://schema.org/name> ?industryName.
    ?gender <http://schema.org/name> ?genderName.
    wdt:P169 <http://schema.org/name> ?rolesName.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender.
    ?company wdt:P452 ?industry;
        wdt:P3320 ?person.

    ?person <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
    ?industry <http://schema.org/name> ?industryName.
    ?gender <http://schema.org/name> ?genderName.
    wdt:P3320 <http://schema.org/name> ?rolesName.
    }
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?companyName
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('companyName', '1&1 Ionos'), ('industryName', 'Internet service provider'), ('rolesName', 'employer'), ('genderName', 'male')]
[('companyName', '1&1 Ionos'), ('industryName', 'Internet service provider'), ('rolesName', 'chief executive officer'), ('genderName', 'male')]
[('companyName', '1-Up Studio'), ('industryName', 'video game industry'), ('rolesName', 'employer'), ('genderName', 'male')]
[('companyName', '100 Worries, LLC'), ('industryName', 'video game'), ('rolesName', 'employer'), ('genderName', 'female')]
[('companyName', '10X Genomics (United States)'), ('industryName', 'biotechnology industry'), ('rolesName', 'employer'), ('genderName', 'male')]
[('companyName', '10X Genomics (United States)'), ('industryName', 'biotechnology industry'), ('rolesName', 'employer'), ('genderName', 'female')]
[('companyName', '11Onze'), ('industryName', 'financial services'), ('rolesName', 'employer'), ('genderName', 'male')]
[('companyName', '11Onze'), ('industryName', 'financial serv

100

In [73]:
# return all the people that works in the German companies minus the German people that works in German companies
queryString = """
SELECT DISTINCT ?personName ?nationalityName
WHERE { 
    ?person wdt:P106 wd:Q2462658;
        wdt:P27 ?nationality.
    ?company wdt:P1037 ?person;
        wdt:P17 wd:Q183.
    
    FILTER(?nationality != wd:Q183).
    ?person <http://schema.org/name> ?personName.
    ?nationality <http://schema.org/name> ?nationalityName.
 
    
} 
ORDER BY ?personName
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Derk Haank'), ('nationalityName', 'Kingdom of the Netherlands')]
[('personName', 'Ferdinand Piëch'), ('nationalityName', 'Federal State of Austria')]
[('personName', 'Ferdinand Piëch'), ('nationalityName', 'Austria')]
[('personName', 'Ferdinand Piëch'), ('nationalityName', 'Nazi Germany')]
[('personName', 'Fernando Carro'), ('nationalityName', 'Spain')]
[('personName', 'Jim Hagemann Snabe'), ('nationalityName', 'Denmark')]
[('personName', 'Lisa Davis'), ('nationalityName', 'United States of America')]
[('personName', 'Melody Harris-Jensbach'), ('nationalityName', 'United States of America')]
[('personName', 'Norbert Rollinger'), ('nationalityName', 'Luxembourg')]
[('personName', 'Parm Sandhu'), ('nationalityName', 'United Kingdom')]
[('personName', 'Pieter Haas'), ('nationalityName', 'Kingdom of the Netherlands')]
[('personName', 'Tuomo Hatakka'), ('nationalityName', 'Finland')]


12

********
# Query 3
Are there german companies with C.E.O., managers, or founders that are not German?

In [52]:
# ask if there exists no German manager (alive) that works for German company
queryString = """
ASK
WHERE {  
    ?person wdt:P106 wd:Q2462658;
        wdt:P27 ?nationality.
    ?company wdt:P1037 ?person;
        wdt:P17 wd:Q183.
        
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
    FILTER(?nationality != wd:Q183).
    ?person <http://schema.org/name> ?personName.
    ?nationality <http://schema.org/name> ?nationalityName.
  
} 
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

> I've taken the information in the initial query. Then I've put a FILTER NOT EXISTS to exclude the death people. Then I've insert the FILTER in order to check if a manager is not german. 
*******

In [23]:
# return the count of the companies grouped by people
queryString = """
SELECT COUNT(?company) ?personName ?person
WHERE {  
    ?person wdt:P106 ?roles. 
    ?company wdt:P169 ?person;
        wdt:P17 wd:Q183.
    
    wdt:P169 <http://schema.org/name> ?roleName.
    ?person <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
} 
GROUP BY ?personName ?person
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '2'), ('personName', 'Thomas Borsch'), ('person', 'http://www.wikidata.org/entity/Q1784696')]
[('callret-0', '1'), ('personName', 'Joachim Wuermeling'), ('person', 'http://www.wikidata.org/entity/Q269632')]
[('callret-0', '2'), ('personName', 'August von Finck'), ('person', 'http://www.wikidata.org/entity/Q104776')]
[('callret-0', '1'), ('personName', 'Sebastian Jurczyk'), ('person', 'http://www.wikidata.org/entity/Q64136415')]
[('callret-0', '2'), ('personName', 'Wolfgang Büchele'), ('person', 'http://www.wikidata.org/entity/Q27022764')]
[('callret-0', '4'), ('personName', 'Carsten Spohr'), ('person', 'http://www.wikidata.org/entity/Q1045681')]
[('callret-0', '2'), ('personName', 'Ludwig Mellinger'), ('person', 'http://www.wikidata.org/entity/Q1874775')]
[('callret-0', '1'), ('personName', 'Mathias Mogge'), ('person', 'http://www.wikidata.org/entity/Q61932890')]
[('callret-0', '1'), ('personName', 'Thomas Winkelmann'), ('person', 'http://www.wikidata.org/entity/

227

In [40]:
# check if the previous query is right
queryString = """
SELECT ?company ?companyName ?personName 
WHERE {  
    wd:Q1784696 wdt:P106 ?roles. 
    ?company wdt:P169 wd:Q1784696;
        wdt:P17 wd:Q183.
    
    wdt:P169 <http://schema.org/name> ?roleName.
    wd:Q1784696 <http://schema.org/name> ?personName.
    ?company <http://schema.org/name> ?companyName.
} 
"""

print("Results")
run_query(queryString)

Results
[('company', 'http://www.wikidata.org/entity/Q163255'), ('companyName', 'Botanic Garden and Botanical Museum Berlin'), ('personName', 'Thomas Borsch')]
[('company', 'http://www.wikidata.org/entity/Q163255'), ('companyName', 'Botanic Garden and Botanical Museum Berlin'), ('personName', 'Thomas Borsch')]


2

In [51]:
# return the predicates derived from roles

queryString = """

    SELECT DISTINCT ?pred ?predName
    WHERE {  
        ?person wdt:P106 ?roles.
        ?roles ?pred ?obj.
        ?company wdt:P3320 ?person;
            wdt:P17 wd:Q183.
       
        
        ?roles <http://schema.org/name> ?rolesName.
        ?pred <http://schema.org/name> ?predName.
        }

"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P3729'), ('predName', 'next lower rank')]
[('pred', 'http://www.wikidata.org/prop/direct/P918'), ('predName', 'NOC Occupation Code')]
[('pred', 'http://www.wikidata.org/prop/direct/P101'), ('predName', 'field of work')]
[('pred', 'http://www.wikidata.org/prop/direct/P1014'), ('predName', 'Art & Architecture Thesaurus ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1036'), ('predName', 'Dewey Decimal Classification')]
[('pred', 'http://www.wikidata.org/prop/direct/P1051'), ('predName', 'PSH ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1225'), ('predName', 'U.S. National Archives Identifier')]
[('pred', 'http://www.wikidata.org/prop/direct/P1245'), ('predName', 'OmegaWiki Defined Meaning')]
[('pred', 'http://www.wikidata.org/prop/direct/P1249'), ('predName', 'time of earliest written record')]
[('pred', 'http://www.wikidata.org/prop/direct/P1256'), ('predName', 'Iconclass notation')]
[('pred', 'http://www.wikidata.org/

214

In [54]:
# return the predicates derived from roles

queryString = """

    SELECT DISTINCT ?obj ?objName
    WHERE {  
        ?person wdt:P106 ?roles.
        ?roles wdt:P108 ?obj.
        ?company wdt:P3320 ?person;
            wdt:P17 wd:Q183.
       
        
        ?roles <http://schema.org/name> ?rolesName.
        ?obj <http://schema.org/name> ?objName.
        }

"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q2990216'), ('objName', 'mining company')]


1

In [ ]:
queryString = """
SELECT (COUNT(?company) AS ?totalCompany) ?personName
WHERE {
    {
    SELECT DISTINCT ?company ?companyName ?personName 
    WHERE {  
        ?person wdt:P106 ?roles. 
        ?company wdt:P169 ?person;
            wdt:P17 wd:Q183.
       
        ?person <http://schema.org/name> ?personName.
        ?company <http://schema.org/name> ?companyName.
        }
    }
} 
GROUP BY ?personName
HAVING (COUNT(?company) >1)
"""

print("Results")
run_query(queryString)

Results
[('totalCompany', '4'), ('personName', 'Andreas Voigt')]
[('totalCompany', '2'), ('personName', 'Theodor Weimer')]
[('totalCompany', '2'), ('personName', 'Torsten Rossmann')]
[('totalCompany', '2'), ('personName', 'Oliver Pommerenke')]
[('totalCompany', '2'), ('personName', 'Ola Källenius')]
[('totalCompany', '2'), ('personName', 'Christoph Minhoff')]
[('totalCompany', '2'), ('personName', 'Tim Steckel')]


7

In [52]:
# restart the workflow for query 4 from here.
#return the count of the companies related to a single person
queryString = """
SELECT DISTINCT ?managerName (COUNT(?company) AS ?howmanycompany)
WHERE { 
                ?manager wdt:P106 wd:Q2462658.
                ?company wdt:P1037|wdt:P169|wdt:P112|wdt:P3320 ?manager.
                ?company wdt:P17 wd:Q183.

                ?manager <http://schema.org/name> ?managerName.
                ?company <http://schema.org/name> ?companyName .
            }
            GROUP BY ?managerName
ORDER BY DESC (?howmanycompany)    

"""

print("Results")
run_query(queryString)

Results
[('managerName', 'Friedrich Joussen'), ('howmanycompany', '3')]
[('managerName', 'Carsten Spohr'), ('howmanycompany', '2')]
[('managerName', 'Stefan Dohler'), ('howmanycompany', '2')]
[('managerName', 'Tobias Bachmüller'), ('howmanycompany', '2')]
[('managerName', 'Hans Riese'), ('howmanycompany', '2')]
[('managerName', 'Sigrid Nikutta'), ('howmanycompany', '2')]
[('managerName', 'Dirk Stenkamp'), ('howmanycompany', '2')]
[('managerName', 'Hans-Gerd Jauch'), ('howmanycompany', '2')]
[('managerName', 'Ola Källenius'), ('howmanycompany', '2')]
[('managerName', 'Gunter Thielen'), ('howmanycompany', '2')]
[('managerName', 'Karl-Heinz Sonne'), ('howmanycompany', '1')]
[('managerName', 'Melanie Kreis'), ('howmanycompany', '1')]
[('managerName', 'Fritz Vahrenholt'), ('howmanycompany', '1')]
[('managerName', 'Tanja Wielgoß'), ('howmanycompany', '1')]
[('managerName', 'Frank Heinricht'), ('howmanycompany', '1')]
[('managerName', 'Frank Sennhenn'), ('howmanycompany', '1')]
[('managerName

134

In [ ]:
# restart the workflow for query 4 from here.
#return the count of the companies related to a single person greater than 1
queryString = """
SELECT ?howmanycompany
WHERE {
        SELECT DISTINCT ?managerName (COUNT(?company) AS ?howmanycompany)
        WHERE { 
            ?manager wdt:P106 wd:Q2462658.
            ?company wdt:P1037|wdt:P169|wdt:P112|wdt:P3320 ?manager.
            ?company wdt:P17 wd:Q183.

            ?manager <http://schema.org/name> ?managerName.
            ?company <http://schema.org/name> ?companyName .
        }
        GROUP BY ?managerName
}
GROUP BY ?howmanycompany
HAVING (?howmanycompany>1)
ORDER BY DESC (?howmanycompany)
"""

print("Results")
run_query(queryString)

**********

# Query 4
Are there people related to multiple german companies?

In [58]:
#return if there is a person related to multiple german companies
queryString = """
ASK WHERE {
        SELECT ?howmanycompany
        WHERE {
                SELECT DISTINCT ?managerName (COUNT(?company) AS ?howmanycompany)
                WHERE { 
                    ?manager wdt:P106 wd:Q2462658.
                    ?company wdt:P1037|wdt:P169|wdt:P112|wdt:P3320 ?manager.
                    ?company wdt:P17 wd:Q183.

                    ?manager <http://schema.org/name> ?managerName.
                    ?company <http://schema.org/name> ?companyName .
                }
                GROUP BY ?managerName
        }
        GROUP BY ?howmanycompany
        HAVING (?howmanycompany>1)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

> I've taken the german company and I've count the number of company related to a single person.
**********

In [38]:
# return information about roles

queryString = """
SELECT DISTINCT ?rolesName ?roles
WHERE { 
    
    ?person wdt:P27 wd:Q183.
    ?person wdt:P106 ?roles.

    
    ?roles <http://schema.org/name> ?rolesName.


} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('rolesName', 'dialect literature'), ('roles', 'http://www.wikidata.org/entity/Q1208422')]
[('rolesName', 'envoy'), ('roles', 'http://www.wikidata.org/entity/Q11051391')]
[('rolesName', 'pedestrian'), ('roles', 'http://www.wikidata.org/entity/Q221488')]
[('rolesName', 'auto mechanic'), ('roles', 'http://www.wikidata.org/entity/Q706835')]
[('rolesName', 'industrial engineer'), ('roles', 'http://www.wikidata.org/entity/Q2267418')]
[('rolesName', 'cardiac surgeon'), ('roles', 'http://www.wikidata.org/entity/Q19842026')]
[('rolesName', 'abbot'), ('roles', 'http://www.wikidata.org/entity/Q103163')]
[('rolesName', 'Minnesänger'), ('roles', 'http://www.wikidata.org/entity/Q11781549')]
[('rolesName', 'ophthalmologist'), ('roles', 'http://www.wikidata.org/entity/Q12013238')]
[('rolesName', 'figure skater'), ('roles', 'http://www.wikidata.org/entity/Q13219587')]
[('rolesName', 'artistic gymnast'), ('roles', 'http://www.wikidata.org/entity/Q13381572')]
[('rolesName', 'canoeist'), ('roles

3115

In [40]:
# return information about literature

queryString = """
SELECT DISTINCT ?predName ?pred ?obj ?objName
WHERE { 
    
    ?person wdt:P27 wd:Q183.
    ?person wdt:P106 wd:Q1208422.
    wd:Q1208422 ?pred ?obj.

    
    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.


} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'instance of'), ('pred', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q223393'), ('objName', 'literary genre')]
[('predName', 'practiced by'), ('pred', 'http://www.wikidata.org/prop/direct/P3095'), ('obj', 'http://www.wikidata.org/entity/Q61043360'), ('objName', 'dialect writer')]
[('predName', 'subclass of'), ('pred', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q8242'), ('objName', 'literature')]


3

In [42]:
# return information about roles manages
#instance of'wdt: P31' wdt:Q28640
queryString = """
SELECT DISTINCT ?predName ?pred ?obj ?objName
WHERE { 
    
    ?person wdt:P27 wd:Q183.
    ?person wdt:P106 wd:Q2462658.
       wd:Q2462658 ?pred ?obj.

    
    ?pred <http://schema.org/name> ?predName.
    ?obj <http://schema.org/name> ?objName.


} 
ORDER BY ?predName
"""

print("Results")
run_query(queryString)

Results
[('predName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('obj', 'http://www.wikidata.org/entity/Q1320883'), ('objName', 'talent manager')]
[('predName', 'different from'), ('pred', 'http://www.wikidata.org/prop/direct/P1889'), ('obj', 'http://www.wikidata.org/entity/Q17128963'), ('objName', 'Manager')]
[('predName', 'field of this occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P425'), ('obj', 'http://www.wikidata.org/entity/Q309100'), ('objName', 'planning')]
[('predName', 'field of this occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P425'), ('obj', 'http://www.wikidata.org/entity/Q2920921'), ('objName', 'management')]
[('predName', 'field of this occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P425'), ('obj', 'http://www.wikidata.org/entity/Q21652209'), ('objName', 'organizational leadership')]
[('predName', 'field of this occupation'), ('pred', 'http://www.wikidata.org/prop/direct/P425'), ('obj', 'http://ww

10

In [44]:
# return the gender and the role of the German person 

queryString = """
SELECT ?roleName ?genderName
WHERE { 
    {
     ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?person wdt:P108 ?company.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person.
  
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P169 ?person.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P3320 ?person.

    }
    ?person <http://schema.org/name> ?personName.
    ?role <http://schema.org/name> ?roleName.
    ?gender <http://schema.org/name> ?genderName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
GROUP BY ?roleName ?genderName
ORDER BY ?roleName
"""

print("Results")
run_query(queryString)

Results
[('roleName', 'Africanist'), ('genderName', 'female')]
[('roleName', 'Africanist'), ('genderName', 'male')]
[('roleName', 'Agriculteur'), ('genderName', 'female')]
[('roleName', 'Amtsarzt'), ('genderName', 'male')]
[('roleName', 'Arabist'), ('genderName', 'female')]
[('roleName', 'Arabist'), ('genderName', 'male')]
[('roleName', 'Armenian Genocide denier'), ('genderName', 'male')]
[('roleName', 'Beamter'), ('genderName', 'male')]
[('roleName', 'Bible translator'), ('genderName', 'female')]
[('roleName', 'Buddhist studies'), ('genderName', 'male')]
[('roleName', 'Catholic priest'), ('genderName', 'male')]
[('roleName', 'Catholic theologian'), ('genderName', 'female')]
[('roleName', 'Catholic theologian'), ('genderName', 'male')]
[('roleName', 'Chief inspector'), ('genderName', 'female')]
[('roleName', 'Commander'), ('genderName', 'male')]
[('roleName', 'County surveyor'), ('genderName', 'male')]
[('roleName', 'Diploma of Business Administration'), ('genderName', 'male')]
[('role

1840

**************
# Query 5.1
How many people for each role and gender are there in Germany?

In [11]:
# return the gender and the role of the German person with the count 

queryString = """
SELECT COUNT(?genderRole) ?genderRole
WHERE { 
    {
     ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?person wdt:P108 ?company.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person.
  
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P169 ?person.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P21 ?gender;
        wdt:P27 wd:Q183.
    ?company wdt:P3320 ?person.

    }
    ?person <http://schema.org/name> ?personName.
    ?role <http://schema.org/name> ?roleName.
    ?gender <http://schema.org/name> ?genderName.
    BIND(CONCAT(?genderName ," - ", ?roleName) AS ?genderRole).
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?genderName
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '14'), ('genderRole', 'female - architectural historian')]
[('callret-0', '49'), ('genderRole', 'female - manager')]
[('callret-0', '55'), ('genderRole', 'female - lawyer')]
[('callret-0', '160'), ('genderRole', 'female - non-fiction writer')]
[('callret-0', '231'), ('genderRole', 'female - author')]
[('callret-0', '1'), ('genderRole', 'female - bartender')]
[('callret-0', '1'), ('genderRole', 'female - skeleton racer')]
[('callret-0', '28'), ('genderRole', 'female - voice actor')]
[('callret-0', '122'), ('genderRole', 'female - classical archaeologist')]
[('callret-0', '2'), ('genderRole', 'female - clarinetist')]
[('callret-0', '3'), ('genderRole', 'female - social worker')]
[('callret-0', '11'), ('genderRole', 'female - neurologist')]
[('callret-0', '3'), ('genderRole', 'female - documentary filmmaker')]
[('callret-0', '1'), ('genderRole', 'female - law professor')]
[('callret-0', '16'), ('genderRole', "female - children's writer")]
[('callret-0', '125'), ('ge

1840

> For this query, I've taken the information in the initial queries.
**********

In [ ]:
#count the people that has important role in a single company
queryString = """
SELECT COUNT(DISTINCT ?person) ?company
WHERE { 
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P1037 ?person.
    }
    UNION
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P169 ?person.
    }
    UNION
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P3320 ?person.
    }
    ?person <http://schema.org/name> ?personName.
    ?role <http://schema.org/name> ?roleName.

} 
GROUP BY ?company
HAVING( COUNT(?person) > 1)
"""

print("Results")
run_query(queryString)

************
# Query 5.2
 Are there companies with multiple german people having important roles?

In [8]:
# return if there are multiple german people having important roles

queryString = """
ASK WHERE{
SELECT COUNT(?company) ?person
WHERE { 
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P1037 ?person.
    }
    UNION
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P169 ?person.
    }
    UNION
    {
        ?person wdt:P106 wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P3320 ?person.
    }
    ?person <http://schema.org/name> ?personName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 

GROUP BY ?person
HAVING( COUNT(?company) > 1)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

****************

# Query 5.3
In which sectors are important german business people working?

In [4]:
# sector of business people (manager/ceo/director)
queryString = """
SELECT DISTINCT ?sectorName
WHERE {
    {
    ?person wdt:P106  wd:Q2462658;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        wdt:P452 ?industry.
    ?industry wdt:P279 ?sector.
    }
    UNION
    {
    ?person wdt:P106 wd:Q2462658;
        wdt:P27 wd:Q183.
    ?company wdt:P169 ?person;
        wdt:P452 ?industry.
    ?industry wdt:P279 ?sector.
    }
    ?sector <http://schema.org/name> ?sectorName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?sectorName
"""

print("Results")
run_query(queryString)

Results
[('sectorName', 'Depository institution')]
[('sectorName', 'activity')]
[('sectorName', 'aerodrome')]
[('sectorName', 'agriculture')]
[('sectorName', 'aviation')]
[('sectorName', 'branch of engineering')]
[('sectorName', 'broadcasting')]
[('sectorName', 'business')]
[('sectorName', 'business service')]
[('sectorName', 'chemical substance')]
[('sectorName', 'commercial building')]
[('sectorName', 'commercial transportation')]
[('sectorName', 'commodity production')]
[('sectorName', 'communication')]
[('sectorName', 'control')]
[('sectorName', 'credit institution')]
[('sectorName', 'defense industry')]
[('sectorName', 'economic activity')]
[('sectorName', 'education')]
[('sectorName', 'electrical industry')]
[('sectorName', 'engineering')]
[('sectorName', 'enterprise')]
[('sectorName', 'extractive industry')]
[('sectorName', 'facility')]
[('sectorName', 'financial intermediary')]
[('sectorName', 'financial product')]
[('sectorName', 'financial services')]
[('sectorName', 'goods a

65

************

In [50]:
# nationality of the people still alive that are not german (manager/ceo/director)
queryString = """
SELECT DISTINCT ?nationalityName
WHERE {
    {
    ?person wdt:P106 ?role;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        wdt:P452 ?industry;
        wdt:P17 ?nationality.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P27 wd:Q183.
    ?company wdt:P169 ?person;
        wdt:P452 ?industry;
        wdt:P27 ?nationality.
    }
    ?person <http://schema.org/name> ?personName.
    ?nationality <http://schema.org/name> ?nationalityName.
    FILTER(?nationality != wd:Q183).
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?objName
"""

print("Results")
run_query(queryString)

Results
[('nationalityName', 'United Kingdom')]
[('nationalityName', 'Brazil')]
[('nationalityName', 'German Democratic Republic')]
[('nationalityName', 'United States of America')]
[('nationalityName', 'Italy')]
[('nationalityName', 'Switzerland')]
[('nationalityName', 'Austria')]
[('nationalityName', 'Poland')]
[('nationalityName', 'Venezuela')]
[('nationalityName', 'Slovenia')]
[('nationalityName', 'Croatia')]
[('nationalityName', 'Hungary')]
[('nationalityName', 'Spain')]
[('nationalityName', 'German Reich')]
[('nationalityName', 'France')]
[('nationalityName', 'Ireland')]
[('nationalityName', 'Belgium')]
[('nationalityName', 'Finland')]
[('nationalityName', 'Netherlands')]
[('nationalityName', 'Grand Duchy of Finland')]


20

In [51]:
# sector of business people (manager/ceo/director)
queryString = """
ASK WHERE {
SELECT DISTINCT ?personName
WHERE {
    {
    ?person wdt:P106 ?role;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        wdt:P452 ?industry;
        wdt:P17 ?nationality.
    }
    UNION
    {
    ?person wdt:P106 ?role;
        wdt:P27 wd:Q183.
    ?company wdt:P169 ?person;
        wdt:P452 ?industry;
        wdt:P27 ?nationality.
    }
    ?person <http://schema.org/name> ?personName.
    ?nationality <http://schema.org/name> ?nationalityName.
    FILTER(?nationality != wd:Q183).
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?objName
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [13]:
# return the predicates connected to company
queryString = """
SELECT DISTINCT ?pred ?pName
WHERE {
    ?person wdt:P106  wd:Q2462658;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        ?pred ?obj.
    
    ?pred <http://schema.org/name> ?pName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('pred', 'http://www.wikidata.org/prop/direct/P2342'), ('pName', 'AGORHA person/institution ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P6295'), ('pName', 'ARTIC artist ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P1661'), ('pName', 'Alexa rank')]
[('pred', 'http://www.wikidata.org/prop/direct/P9618'), ('pName', 'AlternativeTo software ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P6963'), ('pName', 'BaFin Institute ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P2581'), ('pName', 'BabelNet ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3630'), ('pName', 'Babelio author ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P950'), ('pName', 'Biblioteca Nacional de España ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P268'), ('pName', 'Bibliothèque nationale de France ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3377'), ('pName', 'Bloomberg company ID')]
[('pred', 'http://www.wikidata.org/prop/direct/P3888'), ('pName', 'Boijma

193

In [14]:
# return the country connected to company
queryString = """
SELECT DISTINCT ?obj ?oName
WHERE {
    ?person wdt:P106  wd:Q2462658;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        wdt:P17 ?obj.
    
    ?obj <http://schema.org/name> ?oName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
} 
ORDER BY ?oName
"""

print("Results")
run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q40'), ('oName', 'Austria')]
[('obj', 'http://www.wikidata.org/entity/Q142'), ('oName', 'France')]
[('obj', 'http://www.wikidata.org/entity/Q16957'), ('oName', 'German Democratic Republic')]
[('obj', 'http://www.wikidata.org/entity/Q183'), ('oName', 'Germany')]
[('obj', 'http://www.wikidata.org/entity/Q27'), ('oName', 'Ireland')]
[('obj', 'http://www.wikidata.org/entity/Q38'), ('oName', 'Italy')]
[('obj', 'http://www.wikidata.org/entity/Q30'), ('oName', 'United States of America')]


7

In [16]:
# return the country connected to company(not German)
queryString = """
SELECT DISTINCT ?countries ?countriesName
WHERE {
    ?person wdt:P106  wd:Q2462658;
        wdt:P27 wd:Q183.
    ?company wdt:P1037 ?person;
        wdt:P17 ?countries.
    
    ?countries <http://schema.org/name> ?countriesName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
    FILTER NOT EXISTS{?company wdt:P17 wd:Q183}.
} 
ORDER BY ?oName
"""

print("Results")
run_query(queryString)

Results
[('countries', 'http://www.wikidata.org/entity/Q142'), ('countriesName', 'France')]
[('countries', 'http://www.wikidata.org/entity/Q16957'), ('countriesName', 'German Democratic Republic')]
[('countries', 'http://www.wikidata.org/entity/Q30'), ('countriesName', 'United States of America')]
[('countries', 'http://www.wikidata.org/entity/Q40'), ('countriesName', 'Austria')]
[('countries', 'http://www.wikidata.org/entity/Q27'), ('countriesName', 'Ireland')]
[('countries', 'http://www.wikidata.org/entity/Q38'), ('countriesName', 'Italy')]


6

*************
# Query 5.4 
Are there german business people related to non german companies? How many?

In [36]:
# return if there are german business people related to non german companies
queryString = """
ASK WHERE
    { 
    SELECT DISTINCT ?personName 
    WHERE {
        {
        ?person wdt:P106  wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P1037 ?person;
            wdt:P17 ?countries.
        }
        UNION 
        {
        ?person wdt:P106  wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P169 ?person;
            wdt:P17 ?countries.
        }

        ?company <http://schema.org/name> ?companyName.
        ?person <http://schema.org/name> ?personName.
        FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
        FILTER NOT EXISTS{?company wdt:P17 wd:Q183}.
    } 
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

In [38]:
# return the people and the number of companies that they are related to
queryString = """
SELECT (COUNT(DISTINCT ?personName) AS ?howmany)
WHERE {
    {
        ?person wdt:P106  wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P1037 ?person;
            wdt:P17 ?countries.
        }
        UNION 
        {
        ?person wdt:P106  wd:Q2462658;
            wdt:P27 wd:Q183.
        ?company wdt:P169 ?person;
            wdt:P17 ?countries.
        }
    
    
    ?company <http://schema.org/name> ?companyName.
    ?person <http://schema.org/name> ?personName.
    FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
    FILTER NOT EXISTS{?company wdt:P17 wd:Q183}.
} 
ORDER BY DESC (?howmany)
"""

print("Results")
run_query(queryString)

Results
[('howmany', '16')]


1

> For these last queries, I’ve taken the information in the initial query.<br>
> In particular, I’ve checked the predicates connected to the company in order to find something about country. Then I’ve returned only the country that are not German.<br>
> After this operation, I’ve checked for the nationality of the people and found wdt:P27 *country of citizenship* and I’ve restricted the results to the German people. In the output of jupyter there are 16 records, so the correct result is returned.
> In order to retrieve the people still alive, I’ve put this FILTER NOT EXISTS{?person wdt:P20 ?placeOfDeath}.
 ***********